In [28]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [29]:
df=pd.read_csv(r'D:\Data Science ExcelR\Project\co2data.csv')

In [30]:
df.head()

,make,model,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_consumption_city,fuel_consumption_hwy,fuel_consumption_comb(l/100km),fuel_consumption_comb(mpg),co2_emissions
0,ACURA,ILX,COMPACT,2.0,4,AS,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS,Z,12.1,8.7,10.6,27,244


In [31]:
df[['make', 'model', 'vehicle_class']] = df[['make', 'model', 'vehicle_class']].applymap(lambda x: x.lower())
df.head()

,make,model,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_consumption_city,fuel_consumption_hwy,fuel_consumption_comb(l/100km),fuel_consumption_comb(mpg),co2_emissions
0,acura,ilx,compact,2.0,4,AS,Z,9.9,6.7,8.5,33,196
1,acura,ilx,compact,2.4,4,M,Z,11.2,7.7,9.6,29,221
2,acura,ilx hybrid,compact,1.5,4,AV,Z,6.0,5.8,5.9,48,136
3,acura,mdx 4wd,suv - small,3.5,6,AS,Z,12.7,9.1,11.1,25,255
4,acura,rdx awd,suv - small,3.5,6,AS,Z,12.1,8.7,10.6,27,244


In [32]:
df_final=df[["engine_size","transmission", 'fuel_type',"fuel_consumption_comb(mpg)",'co2_emissions']]

In [33]:
df_final.duplicated().sum()

4040

In [34]:
df_final=df_final.drop_duplicates()

In [35]:
df_final.duplicated().sum()

0

In [36]:
# step3: split as X and Y variable
y = df_final["co2_emissions"]
X = df_final[["engine_size","transmission", 'fuel_type',"fuel_consumption_comb(mpg)"]]

## Model Building

In [37]:
X.head()

,engine_size,transmission,fuel_type,fuel_consumption_comb(mpg)
0,2.0,AS,Z,33
1,2.4,M,Z,29
2,1.5,AV,Z,48
3,3.5,AS,Z,25
4,3.5,AS,Z,27


In [38]:
X=pd.get_dummies(X,columns=['fuel_type'], dtype=int)

In [39]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['transmission']=le.fit_transform(X['transmission'])

In [40]:
X.rename(columns={'fuel_type_0':'fuel_type_D','fuel_type_1':'fuel_type_E','fuel_type_2':'fuel_type_N','fuel_type_3':'fuel_type_X','fuel_type_4':'fuel_type_Z'},inplace=True)

In [41]:
X.head(3)

,engine_size,transmission,fuel_consumption_comb(mpg),fuel_type_D,fuel_type_E,fuel_type_N,fuel_type_X,fuel_type_Z
0,2.0,2,33,0,0,0,0,1
1,2.4,4,29,0,0,0,0,1
2,1.5,3,48,0,0,0,0,1


In [42]:
X.columns

Index(['engine_size', 'transmission', 'fuel_consumption_comb(mpg)',
       'fuel_type_D', 'fuel_type_E', 'fuel_type_N', 'fuel_type_X',
       'fuel_type_Z'],
      dtype='object')

In [43]:
X.shape

(3345, 8)

In [44]:
y.shape

(3345,)

In [45]:
# standardization
from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
SS_X = SS.fit_transform(X)
SS_X = pd.DataFrame(SS_X)

In [46]:
# Data partition
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.30,random_state=42)

In [47]:
X_train= SS.fit_transform(X_train)
X_test=SS.fit_transform(X_test)

In [48]:
from sklearn.linear_model import Ridge

In [49]:
ridge_reg=Ridge(alpha=100)
ridge_reg.fit(X_train,Y_train)

Ridge(alpha=100)

In [50]:
Y_pred=ridge_reg.predict(X_test)

In [51]:
import pickle
pickle.dump(ridge_reg,open('model.pkl','wb'))

In [52]:
rr_training=ridge_reg.score(X_train,Y_train)*100
rr_training


91.36716390353483

In [53]:
rr_testing=ridge_reg.score(X_test,Y_test)*100
rr_testing

92.06648681385367